# **LangChain Introduction**

### Introduction to LangChain and it's Expression Language.

In [18]:
#Mistral is used here as example model, which is locally installed through ollama.
from langchain.llms import Ollama

model = Ollama(model="mistral")

mistral.invoke("Tell me a joke about Python!")

" Why did the Python programmer quit his job?\n\nHe didn't get arrays with functions as arguments.\n\n(I'm here all week...)"

### **Chain**

In [27]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Generate a poem about {topic}")
chain = prompt | model
chain

ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='Generate a poem about {topic}'))])
| Ollama(model='mistral')

In [28]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [29]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [30]:
model.input_schema.schema()

{'title': 'OllamaInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},

In [37]:
chain.output_schema.schema()

{'title': 'OllamaOutput', 'type': 'string'}

In [42]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are computer science professor from world class university."),
    ("user", "{input}")])
chain2 = prompt2 | model

In [43]:
chain2.invoke({"input": "what is rust mainly used for?"})

" As a world-class computer science professor, I'd be happy to help answer your question about Rust. Rust is a multi-paradigm programming language that is designed to provide better memory safety while maintaining high performance. It was developed by Mozilla Research with the primary goal of addressing the common pitfalls of C++: memory safety and threading without data races.\n\nRust is mainly used for system programming, game development, and building high-performance applications where control over resources and efficient use of hardware are essential. Its strong type system, zero-cost abstractions, and ownership model enable developers to write reliable code with minimal runtime overhead. This makes it an excellent choice for projects that require a balance between performance, safety, and maintainability.\n\nSome popular use cases for Rust include:\n\n1. System programming: With its memory safety features and low-level control, Rust is ideal for developing system software such as

In [44]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain2 = prompt2 | model | output_parser

In [45]:
chain2.invoke({"input": "what is rust mainly used for?"})

" As a computer science professor from a world-class university, I can tell you that Rust is a multi-paradigm programming language known for its focus on performance, safety, and concurrency. While it can be used for various purposes, Rust is particularly popular among system programmers and developers who need high performance with memory safety guarantees. Here are some common use cases:\n\n1. Systems programming: Rust's strong type system, ownership model, and zero-cost abstractions make it a great choice for developing system software and low-level applications where memory management is critical.\n\n2. Concurrent programming: With built-in support for concurrency through its 'async-await' feature, Rust makes it easier to write safe concurrent code without the need for complex locks or other synchronization mechanisms.\n\n3. Game development: Many game engines, such as Amethyst and Rusty Engine, have been developed in Rust due to its low-level control, performance capabilities, and

### **Streaming responses**

In [26]:
pizza = chain.stream({"topic": "Pizza"})
for content in pizza:
    print(content, end="", flush=True)

 In the heart of the kitchen, a tale unfolds,
A story of joy, where love and art collide,
A symphony of flavors in perfect harmony,
Born from humble roots, this delightful spree.

Picture this scene, beneath the twinkling light,
Of a pizza pie, bathed in gentle sight,
An canvas blank, ready for a master's touch,
With sauce as palette and cheese to hutch.

Golden crust, a stage for delectable feast,
Tomato's dance atop a bed of creamy beest,
Mozzarella dreams in milky, tender bloom,
Pepperoni soldiers, waiting for the room.

A garden of vegetables, each vibrant and bright,
Bell peppers, onions, olives, an exquisite sight,
With a dash of herbs, basil's sweet, fragrant allure,
An aromatic journey, that leaves us wanting more.

The oven roars to life with a fiery might,
Kissing each ingredient into a perfect delight,
Golden brown perfection, ready to be devoured,
A moment of bliss, as we savor each power.

Pizza pie, you've woven a magical spell,
Bringing us together, as we tell stories we

### **Parallel Processing**

In [32]:
#allow each element to run in parallel
from langchain.schema.runnable import RunnableParallel

chain1 = ChatPromptTemplate.from_template("Tell a joke about {topic}") | model
chain2 = ChatPromptTemplate.from_template("Write a short explanation about {topic}") | model
combined = RunnableParallel(joke=chain1, exp=chain2)

In [33]:
%%time

chain1.batch([{"topic": "Science"}, {"topic": "Motorcycle"}])

CPU times: user 31.1 ms, sys: 15.7 ms, total: 46.8 ms
Wall time: 5 s


[' Why did the biology textbook feel hated?\n\nBecause people kept spouting things at it! (Sputting = Spewing + Putting) But in all seriousness, why were the scientists studying the old mummy in the lab so excited? Because they found out it was a mumm-yield! (Yield = result or discovery) Science can be full of puns!',
 ' Why don\'t motorcycles ever forget where they parked?\n\nBecause they always find their own spot!\n\nOr how about this one:\n\nWhy did the biker name his hog "Rusty"?\n\nBecause he was always fixing it up!\n\nAnd for a classic, why did the motorcycle fall over? Because it saw the salad dressing! (Remember, this is just a joke and not an actual reason why a motorcycle would fall over)']

In [35]:
%%time

chain2.batch([{"topic": "Science"}, {"topic": "Motorcycle"}])

CPU times: user 89.3 ms, sys: 1.95 ms, total: 91.2 ms
Wall time: 10 s


[" Science is a systematic and logical method of discovering knowledge about the physical world around us. It involves making observations, asking questions, formulating hypotheses (possible answers), testing these hypotheses through experiments, and analyzing data to draw conclusions. Science is based on evidence, and its methods are designed to minimize errors and biases. The ultimate goal of science is to expand our understanding of the universe, from the smallest subatomic particles to the vastness of the cosmos. Science is continually evolving as new discoveries and technologies push the boundaries of our knowledge. It's an essential aspect of modern society, influencing many aspects of our lives, including healthcare, technology, and infrastructure development.",
 ' A motorcycle is a two- or three-wheeled vehicle that uses an internal combustion engine or an electric motor to propel the rider and a passenger (if equipped) forward. The design, construction, and specifications of m

In [36]:
%%time

combined.batch([{"topic": "Sea"}, {"topic": "Car"}])

CPU times: user 92.1 ms, sys: 15.6 ms, total: 108 ms
Wall time: 11 s


[{'joke': ' Why don\'t fish sing in the bath?\n\nBecause it might give them reef-ion! This one is a play on words, as "reef" is a type of underwater structure and "refection" is a synonym for reflection. But in the context of singing, we can imagine "re-fision" which would be fishy business indeed!',
  'exp': " A sea is a large body of saline water that is surrounded by land. It covers about 71% of the Earth's surface and plays a vital role in many aspects of life. Seas provide habitat for various forms of marine life, including fish, crustaceans, mollusks, and other organisms. They also serve as crucial sources of food for billions of people worldwide.\n\nMoreover, seas influence the Earth's climate by absorbing and releasing heat, and they contribute significantly to the planet's carbon cycle. Seas are essential for various human activities such as shipping, tourism, and energy production through sources like oil and natural gas. The five largest seas in the world are the Atlantic Oc